In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [ ]:
# 1 טעינת סט הנתונים Yeast
yeast = fetch_openml(name="yeast", version=4, as_frame=True)
X = yeast.data

y = yeast.target.eq('TRUE').astype(int)

In [ ]:
#check the dtype and info about X and y, see what you are working with

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Att94,Att95,Att96,Att97,Att98,Att99,Att100,Att101,Att102,Att103
0,0.004168,-0.170975,-0.156748,-0.142151,0.058781,0.026851,0.197719,0.041850,0.066938,-0.056617,...,0.006166,-0.012976,-0.014259,-0.015024,-0.010747,0.000411,-0.032056,-0.018312,0.030126,0.124722
1,-0.103956,0.011879,-0.098986,-0.054501,-0.007970,0.049113,-0.030580,-0.077933,-0.080529,-0.016267,...,0.007680,0.027719,-0.085811,0.111123,0.050541,0.027565,-0.063569,-0.041471,-0.079758,0.017161
2,0.509949,0.401709,0.293799,0.087714,0.011686,-0.006411,-0.006255,0.013646,-0.040666,-0.024447,...,0.096277,-0.044932,-0.089470,-0.009162,-0.012010,0.308378,-0.028053,0.026710,-0.066565,-0.122352
3,0.119092,0.004412,-0.002262,0.072254,0.044512,-0.051467,0.074686,-0.007670,0.079438,0.062184,...,-0.083809,0.200354,-0.075716,0.196605,0.152758,-0.028484,-0.074207,-0.089227,-0.049913,-0.043893
4,0.042037,0.007054,-0.069483,0.081015,-0.048207,0.089446,-0.004947,0.064456,-0.133387,0.068878,...,-0.060467,0.044351,-0.057209,0.028047,0.029661,-0.050026,0.023248,-0.061539,-0.035160,0.067834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412,-0.119784,0.001259,-0.123645,-0.015513,-0.059683,0.091032,-0.043302,0.229219,-0.071498,0.182709,...,0.024084,-0.055915,-0.055593,-0.049642,0.018571,0.068742,-0.061001,-0.081132,-0.065844,0.001267
2413,0.085327,0.058590,0.085268,-0.020897,0.068972,0.030125,0.078056,0.011346,0.052618,0.066093,...,-0.079992,-0.075444,0.294987,-0.076379,-0.076293,-0.072451,-0.052258,-0.040026,0.342176,-0.169668
2414,0.082526,-0.095571,-0.022019,-0.046793,-0.038360,0.041084,0.056509,0.011749,-0.029657,-0.012198,...,-0.006624,-0.036850,-0.064831,-0.068696,-0.068521,-0.039841,0.274575,-0.066957,0.260121,-0.125303
2415,-0.130830,0.008868,-0.009457,-0.058930,-0.041224,0.042269,0.117717,0.037388,-0.085563,0.136649,...,0.085087,0.033166,-0.012710,0.135359,0.213512,-0.107561,-0.081925,-0.122332,-0.022453,0.001953


In [ ]:
# 2 פיצול לנתוני אימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 3 נורמליזציה של הנתונים
scaler = StandardScaler()
X_train, X_test =scaler.fit_transform(X_train), scaler.transform(X_test)

In [ ]:
# 4 המרת הנתונים ל-Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
# 5 יצירת המודל
input_dim = X.shape[1]  # מספר הפיצ'רים
num_labels = y.shape[1]  # מספר המחלקות
model = nn.Sequential(nn.Linear(input_dim,512),
                      nn.ReLU(),
                      nn.Dropout(),
                      nn.Linear(512,128),
                      nn.ReLU(),
                      nn.Dropout(),
                      nn.Linear(128,num_labels),
                     )

In [ ]:
# 6 הגדרת פונקציית הפסד ואופטימיזציה
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 7 אימון המודל
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}")

Epoch [0/1000], Loss: 0.00312688946723938
Epoch [100/1000], Loss: 0.00010626946459524333
Epoch [200/1000], Loss: 4.9730359023669735e-05
Epoch [300/1000], Loss: 3.1316947570303455e-05
Epoch [400/1000], Loss: 2.236948967038188e-05
Epoch [500/1000], Loss: 1.705893373582512e-05
Epoch [600/1000], Loss: 1.3588537512987386e-05
Epoch [700/1000], Loss: 1.1155773790960666e-05
Epoch [800/1000], Loss: 9.35623756959103e-06
Epoch [900/1000], Loss: 7.973648280312773e-06


In [ ]:
# חיזוי על נתוני הבדיקה
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = (test_outputs > 0.5).float()  # הפיכת הסתברויות ל-0/1

# יצירת דוח ביצועים מלא
report = classification_report(y_test_tensor.numpy(),
                               predictions.numpy(),
                               target_names=[f"Label_{i+1}" for i in range(y_test_tensor.shape[1])])

# הצגת הדוח
print(report)

              precision    recall  f1-score   support

     Label_1       0.68      0.52      0.58       157
     Label_2       0.59      0.57      0.58       214
     Label_3       0.64      0.72      0.67       190
     Label_4       0.62      0.61      0.61       166
     Label_5       0.54      0.47      0.51       131
     Label_6       0.51      0.37      0.43       111
     Label_7       0.46      0.29      0.36        78
     Label_8       0.39      0.18      0.25        87
     Label_9       0.12      0.06      0.08        33
    Label_10       0.53      0.14      0.22        59
    Label_11       0.40      0.11      0.18        70
    Label_12       0.72      0.94      0.81       341
    Label_13       0.71      0.90      0.79       338
    Label_14       0.00      0.00      0.00         2

   micro avg       0.64      0.62      0.63      1977
   macro avg       0.49      0.42      0.43      1977
weighted avg       0.61      0.62      0.60      1977
 samples avg       0.65   

C:\Users\USER\anaconda3\envs\env23\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
